# Introducción al Aprendizaje Automático.

## Validación del modelo

### Qué es la validación del modelo

En esta lección, aprenderemos a usar la validación del modelo para medir su calidad. Medir la calidad del modelo es la clave para mejorar iterativamente nuestros modelos.

Querremos evaluar casi todos los modelos que hayamos construido. En la mayoría de las aplicaciones (aunque no todas), la medida relevante de la calidad del modelo es la precisión predictiva. En otras palabras, cuán cerca están las predicciones del modelo de lo que realmente sucede.

Muchas personas cometen un gran error al medir la precisión predictiva. Hacen predicciones con sus datos de *entrenamiento* y comparan esas predicciones con los valores objetivo en los datos de *entrenamiento*. Veremos el problema con este enfoque y cómo resolverlo en un momento, pero pensemos cómo lo haríamos primero.

Primero debe resumir la calidad del modelo de una manera comprensible. Si compara los valores predichos y reales de 10.000 casas, probablemente encontrará una combinación de buenas y malas predicciones. Mirar a través de una lista de 10.000 valores predichos y reales no tendría sentido. Necesitamos resumir esto en una sola métrica.

Hay muchas métricas para resumir la calidad del modelo, pero comenzaremos con una llamada **Error Absoluto Medio**, *Mean Absolute Error*, (también llamada MAE). Analicemos esta métrica comenzando con la primera palabra, error.

El error de predicción para cada casa es:

`error=actual−predicción`

Entonces, si una casa cuesta 150.000 dólares y predice que costará 100.000 dólares, el error es 50.000 dólares.

Con la métrica MAE, tomamos el valor absoluto de cada error. Esto convierte cada error en un número positivo. Luego tomamos el promedio de esos errores absolutos. Esta es nuestra medida de calidad del modelo. En lenguaje natural se puede decir como

> De media, nuestras predicciones se desvían aproximadamente en X

Para calcular el MAE, primero necesitamos un modelo.

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeRegressor

# Cargamos los datos
melbourne_file_path = "./input/melbourne-housing-snapshot/melb_data.csv"
melbourne_data = pd.read_csv(melbourne_file_path) 
# Eliminamos las filas con valores de precio ausentes
filtered_melbourne_data = melbourne_data.dropna(axis=0)
# Elegimos el objetivo y las características
y = filtered_melbourne_data.Price
melbourne_features = ['Rooms', 'Bathroom', 'Landsize', 'BuildingArea', 
                        'YearBuilt', 'Lattitude', 'Longtitude']
X = filtered_melbourne_data[melbourne_features]

# Definimos modelo
melbourne_model = DecisionTreeRegressor()
# Entrenamos modelo
melbourne_model.fit(X, y)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='best')

Una vez que tenemos un modelo, así es como calculamos el error absoluto medio:

In [2]:
from sklearn.metrics import mean_absolute_error

predicted_home_prices = melbourne_model.predict(X)
mean_absolute_error(y, predicted_home_prices)

434.71594577146544

### El problema con las puntuaciones "en muestras"

La medida que acabamos de calcular se puede llamar una puntuación "en muestra". Utilizamos una sola "muestra" de casas para construir el modelo y evaluarlo. He aquí por qué esto es malo. 

Imaginemos que, en el gran mercado inmobiliario, el color de la puerta no está relacionado con el precio de la vivienda. Sin embargo, en la muestra de datos que usamos para construir el modelo, todas las casas con puertas verdes eran muy caras. El trabajo del modelo es encontrar patrones que pronostiquen los precios de las viviendas, por lo que veremos este patrón y siempre predeciremos los altos precios de las viviendas con puertas verdes.

Dado que este patrón se derivó de los datos de entrenamiento, el modelo aparecerá exacto en los datos de entrenamiento. Pero si este patrón no se cumple cuando el modelo ve nuevos datos, el modelo sería muy inexacto cuando se usa en la práctica.

Dado que el valor práctico de los modelos proviene de hacer predicciones sobre nuevos datos, medimos el rendimiento de los datos que no se utilizaron para construir el modelo. La forma más directa de hacer esto es excluir algunos datos del proceso de construcción del modelo y luego usarlos para probar la precisión del modelo en datos que no ha visto antes. Estos datos se denominan **datos de validación**.

### Codifiquémoslo

La biblioteca scikit-learn tiene una función `train_test_split` para dividir los datos en dos partes. Usaremos algunos de esos datos como datos de entrenamiento para entrenar el modelo, y usaremos los otros datos como datos de validación para calcular el `mean_absolute_error`.

Aquí está el código:

In [3]:
from sklearn.model_selection import train_test_split

# Dividimos los datos en datos de entrenamiento y de validación, tanto para las características como para el objetivo
# La división está basada en un número generado aleatoriamente. Proporcionando un valor numérico al argumento
# random_state garantizamos que obtendremos la misma división cada vez que ejecutemos este script
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 0)
# Definir modelo
melbourne_model = DecisionTreeRegressor()
# Entrenar model
melbourne_model.fit(train_X, train_y)

# obtenemos la predicción de precios sobre los datos de validación
val_predictions = melbourne_model.predict(val_X)
print(mean_absolute_error(val_y, val_predictions))

265004.8437701743



¡Ups! Nuestro error absoluto medio para los datos "en muestra" fue de aproximadamente 500 dólares. Fuera de la muestra es más de 250.000 dólares.

Esta es la diferencia entre un modelo que es casi exactamente correcto y uno que no se puede usar para la mayoría de los propósitos prácticos. Como punto de referencia, el valor promedio de una vivienda en los datos de validación es de 1,1 millones de dólares. Entonces, el error en los nuevos datos es aproximadamente una cuarta parte del valor promedio de una vivienda.

Hay muchas formas de mejorar este modelo, como experimentar para encontrar mejores características o diferentes tipos de modelos.

## Ejercicio

## Resumen

Hemos construido un modelo. En este ejercicio probaremos cuán bueno es.

In [4]:
import pandas as pd
from sklearn.tree import DecisionTreeRegressor

# Path del archivo a leer
iowa_file_path = "./input/melbourne-housing-snapshot/train.csv"

home_data = pd.read_csv(iowa_file_path)
y = home_data.SalePrice
feature_columns = ['LotArea', 'YearBuilt', '1stFlrSF', '2ndFlrSF', 'FullBath', 'BedroomAbvGr', 'TotRmsAbvGrd']
X = home_data[feature_columns]

# Especifica Modelo
iowa_model = DecisionTreeRegressor()
# Entrena Modelo
iowa_model.fit(X, y)

print('Primeras prediciones "en muestra":', iowa_model.predict(X.head()))
print("Valores actuales del objetivos para esas casas:", y.head().tolist())

Primeras prediciones "en muestra": [208500. 181500. 223500. 140000. 250000.]
Valores actuales del objetivos para esas casas: [208500, 181500, 223500, 140000, 250000]


### Paso 1: Divide tus datos

Usa la función `train_test_split` para dividir sus datos.

Recuerda, tus características se cargan en el DataFrame **X** y su objetivo se carga en **y**.

In [5]:
from sklearn.model_selection import train_test_split

train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 1)

### Paso 2: Especifica y entrena un modelo

Crea un modelo `DecisionTreeRegressor` y entrénalo.

In [6]:
# Especifica Modelo
iowa_model = DecisionTreeRegressor(random_state=1)
# Entrena Modelo
iowa_model.fit(train_X, train_y)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=1, splitter='best')

### Paso 3: Haz predicciones con los datos de validación

In [7]:
val_predictions = iowa_model.predict(val_X)

Inspecciona tus predicciones y valores reales con los datos de validación.

In [8]:
# print the top few validation predictions
print('Primeras prediciones sobre datos de validación:', iowa_model.predict(val_X.head()))
# print the top few actual prices from validation data
print("Valores actuales del objetivos para esas casas:", val_y.head().tolist())

Primeras prediciones sobre datos de validación: [186500. 184000. 130000.  92000. 164500.]
Valores actuales del objetivos para esas casas: [231500, 179500, 122000, 84500, 142000]


## Paso 4: Calcular el error absoluto medio en los datos de validación

In [9]:
from sklearn.metrics import mean_absolute_error
val_mae = mean_absolute_error(val_y, val_predictions)

val_mae

29652.931506849316

¿Es bueno ese MAE? No existe una regla general sobre qué valores son buenos que se aplique a todas las aplicaciones. Pero veremos cómo usar (y mejorar) este número más adelante.